In [1]:
#SET UP
#import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel

In [16]:
#PROCESS INPUTS
df = pd.read_csv('/home/j/temp/fed1/SEER_case_analysis_formatted_add_unknown_to_other_mortality_with_age_specific_parameters_n=10_slope_off10_4_20_21.csv')

#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]
df = df[df['slope']<0]

df.head()

,Unnamed: 0,cause_name,age_name,age_group_id,V1,cases,deaths,mi_ratio,five_year_cs_survival,five_year_observed_survival,...,year_id,sex_id,acause,age,sex,slope,slope_SE,slope_upper,slope_lower,disease_period
0,1,Acute Lymphocytic Leukemia,00-04 years,1,1,6860,327.0,0.047668,0.932799,0.924490,...,2000-2017,3,Acute Lymphocytic Leukemia,0,3,-0.280916,0.024726,-0.231464,-0.330368,10
1,2,Acute Lymphocytic Leukemia,05-09 years,6,49,3656,344.0,0.094092,0.935722,0.926696,...,2000-2017,3,Acute Lymphocytic Leukemia,1,3,-0.165739,0.021447,-0.122844,-0.208633,10
2,3,Acute Lymphocytic Leukemia,10-14 years,7,99,2364,381.0,0.161168,0.858291,0.840102,...,2000-2017,3,Acute Lymphocytic Leukemia,2,3,-0.271438,0.018503,-0.234432,-0.308443,10
3,4,Acute Lymphocytic Leukemia,15-19 years,8,160,1913,470.0,0.245687,0.769472,0.740721,...,2000-2017,3,Acute Lymphocytic Leukemia,3,3,-0.312613,0.022091,-0.268430,-0.356795,10
4,5,Acute Lymphocytic Leukemia,20-24 years,9,220,1056,486.0,0.460227,0.615530,0.570076,...,2000-2017,3,Acute Lymphocytic Leukemia,4,3,-0.354939,0.027534,-0.299871,-0.410008,10


In [17]:
#RUN MODEL
num_years = 5

model = DecayMIRModel(df['mi_ratio'],
                 df['other_mortality'],
                 df['disease_period'],
                 df['slope'])
model.compute_base_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df['base_excess_mortality'] = model.base_excess_mortality
df['abs_survival_rate'] = survival_rate['abs']
df['rel_survival_rate'] = survival_rate['rel']

In [18]:
df.head()

,Unnamed: 0,cause_name,age_name,age_group_id,V1,cases,deaths,mi_ratio,five_year_cs_survival,five_year_observed_survival,...,age,sex,slope,slope_SE,slope_upper,slope_lower,disease_period,base_excess_mortality,abs_survival_rate,rel_survival_rate
0,1,Acute Lymphocytic Leukemia,00-04 years,1,1,6860,327.0,0.047668,0.932799,0.924490,...,0,3,-0.280916,0.024726,-0.231464,-0.330368,10,0.017269,0.952669,0.964531
1,2,Acute Lymphocytic Leukemia,05-09 years,6,49,3656,344.0,0.094092,0.935722,0.926696,...,1,3,-0.165739,0.021447,-0.122844,-0.208633,10,0.023237,0.917231,0.938956
2,3,Acute Lymphocytic Leukemia,10-14 years,7,99,2364,381.0,0.161168,0.858291,0.840102,...,2,3,-0.271438,0.018503,-0.234432,-0.308443,10,0.059864,0.861846,0.878616
3,4,Acute Lymphocytic Leukemia,15-19 years,8,160,1913,470.0,0.245687,0.769472,0.740721,...,3,3,-0.312613,0.022091,-0.268430,-0.356795,10,0.110294,0.763290,0.801562
4,5,Acute Lymphocytic Leukemia,20-24 years,9,220,1056,486.0,0.460227,0.615530,0.570076,...,4,3,-0.354939,0.027534,-0.299871,-0.410008,10,0.266854,0.562014,0.599868


In [19]:
df.to_csv('/home/j/temp/fed1/testing_new_decay_survival_model_outputs_on_case_analysis_data_cancer_specific_age_specific_slopes_n=10_slope_off10.csv')